In [ ]:
#import zipfile
import numpy as np
import pandas
import os
import io
from pprint import pprint
import json
from datetime import datetime
print('imported')

In [ ]:
derivedfiles = []

#yeararray = np.linspace(1921,2016,96)
yeararray = []
for i in np.linspace(1942,1959,18):
    yeararray.append(i)
for i in np.linspace(1962,1968,7):
    yeararray.append(i)
for i in np.linspace(1971,1973,3):
    yeararray.append(i)
#print(yeararray)

for oneyear in yeararray:
    for itsleague in ['A', 'N']:
        afile = 'retrosheet/' + '{:.0f}'.format(oneyear)+'.ED'+itsleague
        if (os.path.isfile(afile)):
            derivedfiles.append(afile)

print(derivedfiles)
#print('Derived game files identified')

In [ ]:
derivedgames = []
gamecount = -1

for filename in derivedfiles:
#    print(filename)
    for line in io.open(filename,encoding='utf-8').readlines():
        linearray = line.split(',')
#        print(linearray)
        linearray[-1] = linearray[-1][0:-1]  # cut off newline character
        if (linearray[0] == 'id'):
            # 'id' starts the next game, so immediately advance gamecount  
            # record prior game data by adding it to the gamelogs list, unless gamecount = -1
            if (gamecount >= 0):
                thisgamedict['info'] = info_dict
                thisgamedict['visiting_starters'] = vis_start_list.tolist()
                thisgamedict['home_starters'] = home_start_list.tolist()
                thisgamedict['events'] = events.tolist()
                thisgamedict['data'] = data_list.tolist()
                # append the dictionary describing this game to the list of games
                derivedgames.append(thisgamedict)

            # advance the gamecount and initialize a new game
            gamecount = gamecount + 1
            thisgamedict = {}
            info_dict = {}
            start_dict = {}
            vis_start_list = np.empty((0,4),dtype=object)
            home_start_list = np.empty((0,4),dtype=object)
            vis_sub_list = np.empty((0,4),dtype=object)
            home_sub_list = np.empty((0,4),dtype=object)
            events = np.empty((0,12),dtype=object)
            data_list = np.empty((0,2),dtype=object)
            eventnum = 1

            # the first thing to go into the new game is the game id.
            # the [0:-1] is there to cut off the newline character
            thisgamedict['id'] = linearray[1]

        elif (linearray[0] == 'version'):
            thisgamedict['version'] = linearray[1]
        elif (linearray[0] == 'info'):
            info_dict[linearray[1]] = linearray[2]
        elif (linearray[0] == 'start'):
            thisbatter = np.empty((1,4),dtype=object)
            if (linearray[3] == '0'):
                thisbatter[0,0] = linearray[1]
                thisbatter[0,1] = linearray[2]
                thisbatter[0,2] = linearray[4]
                thisbatter[0,3] = linearray[5]
                vis_start_list = np.vstack((vis_start_list, thisbatter))
            elif (linearray[3] == '1'):
                thisbatter[0,0] = linearray[1]
                thisbatter[0,1] = linearray[2]
                thisbatter[0,2] = linearray[4]
                thisbatter[0,3] = linearray[5]
                home_start_list = np.vstack((home_start_list, thisbatter))
            else:
                print('NEITHER VISITOR NOR HOME????')
                
        elif (linearray[0] == 'data'):
            thisdata = np.empty((2),dtype=object)
            thisdata[0] = linearray[2]
            thisdata[1] = linearray[3]
            data_list = np.vstack((data_list,thisdata))
        else:
#            print(linearray)
            thisevent = []
            thisevent.append(eventnum)
            thisevent.append(linearray[0])  # say what type of event: play/com/sub/etc.
            if (linearray[0] == 'play'):
                for i in range(1,7):
                    thisevent.append(linearray[i])
                for i in range(8,12):   # comment, sub_batting_order, sub_position empty
                    thisevent.append('')
            elif (linearray[0] == 'com'):
                for i in range(1,7):
                    thisevent.append('')
                thisevent.append(linearray[1])
                for i in range(9,12):
                    thisevent.append('')
            elif (linearray[0] == 'sub'):
                thisevent.append('')    # inning, fill in later
                thisevent.append('')    # batting team (vis/home), fill in later
                thisevent.append(linearray[1])  # playerID of player coming in
                for i in range(4,8):
                    thisevent.append('')  # count/pitches/result/comment all blank
                thisevent.append(linearray[3]) # batting or fielding sub
                thisevent.append(linearray[4]) # sub's batting order
                thisevent.append(linearray[5]) # sub's fielding position
            elif (linearray[0] == 'badj'):
#                badj_games.append(thisgamedict['id'])
                thisevent.append('')    # inning, fill in later
                thisevent.append('')  # visitor/home, fill in later
                thisevent.append(linearray[1])  # playerID of adjusting batter                
                for i in range(4,7):
                    thisevent.append('')  # count/pitches/result/comment all blank
                thisevent.append(linearray[0]+": "+linearray[2]) # comment should say: "badj: {hand adjusted to}"
                for i in range(8,11):
                    thisevent.append('')    # sub-related variables are empty
            elif (linearray[0] == 'padj'):
#                padj_games.append(thisgamedict['id'])
                thisevent.append('')    # inning, fill in later
                thisevent.append('')  # visitor/home, fill in later
                thisevent.append(linearray[1])  # playerID of adjusting batter                
                for i in range(4,7):
                    thisevent.append('')  # count/pitches/result/comment all blank
                thisevent.append(linearray[0]+": "+linearray[2]) # comment should say: "padj: {hand adjusted to}"
                for i in range(8,11):
                    thisevent.append('')    # sub-related variables are empty
            elif (linearray[0] == 'ladj'):
#                ladj_games.append(thisgamedict['id'])
                thisevent.append('')    # inning, fill in later
                thisevent.append(linearray[1])  # visitor/home
                for i in range(3,7):
                    thisevent.append('')  # playerID/count/pitches/result/comment all blank
                thisevent.append(linearray[0]+": "+linearray[2]) # comment should say: "ladj: {whatever}"
                thisevent.append(linearray[1])  # visitor/home - team making adjustment
                for i in range(9,11):
                    thisevent.append('')    # sub-related variables are empty
            else:
                print(linearray)
            events = np.vstack((events,thisevent))
            eventnum = eventnum + 1

thisgamedict['info'] = info_dict
thisgamedict['visiting_starters'] = vis_start_list.tolist()
thisgamedict['home_starters'] = home_start_list.tolist()
thisgamedict['events'] = events.tolist()
thisgamedict['data'] = data_list.tolist()
# append the dictionary describing this game to the list of games
derivedgames.append(thisgamedict)

print('{0:,.0f} derived games found and processed'.format(len(derivedgames)))

with open('derived_events.json', 'w') as f:
    json.dump(allevents, f)
print('Derived events file saved')

In [ ]:
allgames_df = pandas.read_csv('gamedata.csv', low_memory=False)
#allgames_df = allgames_df.set_index('gameID')
allgames_df['date'] = pandas.to_datetime(allgames_df['date'], format='%Y%m%d', errors='coerce')

#allgames_df.head(1)[0:20].transpose()
print('Loaded original list of games (n={0:,.0f})'.format(len(allgames_df)))
allgames_df.index.name = 'rawID'
allgames_df['gameID'] = allgames_df['home'].astype(str) + allgames_df['date'].apply(lambda x: x.strftime('%Y%m%d')) + allgames_df['gameno'].astype(str)
#allgames_df
print('Calculated each gameID for comparison')
allgames = allgames_df['gameID'].tolist()
#print(allgames[0:10])
print('Saved list of all games (n={0:,.0f})'.format(len(allgames)))

In [ ]:
with open('raw/events_raw_1942.json','r') as f:
    allevents = json.load(f)
print('Loaded events data for {0:,.0f} events'.format(len(allevents)))

In [ ]:
# All derived games appear in the game list, but none appear in the current events list. That is as it should be.
allevents_df = pandas.DataFrame(allevents)
derivedgames_df = pandas.DataFrame(derivedgames)
print('Derived games that appear in the full games list: {0:,.0f}'.format(len(allgames_df[allgames_df['gameID'].isin(derivedgames_df['id'])])))
print('Derived games that already appeared in the events list: {0:,.0f}'.format(len(allevents_df[allevents_df['id'].isin(derivedgames_df['id'])])))

In [ ]:
print('Previously, events data includes events from {0:,.0f} games.'.format(len(allevents)))
allevents = allevents + derivedgames
#derivedgames_df.head(1)
print('Now, events data includes events from {0:,.0f} games.'.format(len(allevents)))

In [ ]:
with open('with_derived/events_with_derived_1970.json', 'w') as f:
    json.dump(allevents, f)
print('Events with derived events file saved')